<a href="https://colab.research.google.com/github/ipavlopoulos/xlmr-emotion/blob/main/xlmr_e_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#@title Source Code
%%capture
!pip install transformers[sentencepiece]

#import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input,Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.random import set_seed
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# Load the TensorBoard notebook extension.
%load_ext tensorboard
from datetime import datetime
from packaging import version
from tensorflow import keras
#from google_trans_new import google_translator

# Define the model architecture (same as before)
def build_model(transformer, max_len):
    np.random.seed(2909)
    set_seed(1995)

    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")

    # Define the Lambda layer with output_shape
    def transformer_wrapper(inputs):
        return transformer(inputs)[0]

    # Get the shape of the output tensor from the transformer model
    output_shape = (max_len, transformer.config.hidden_size)  # (sequence_length, hidden_size)

    sequence_output = Lambda(transformer_wrapper, output_shape=output_shape)(input_word_ids)
    cls_token = sequence_output[:, 0, :]  # Extract the [CLS] token representation
    out = Dense(8, activation='sigmoid')(cls_token)  # Output layer

    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Model configuration
model_name = "jplu/tf-xlm-roberta-base"

# Instantiate the transformer model
transformer_layer = TFAutoModel.from_pretrained(model_name)

In [30]:
#@title Download the weights
# downloading the weights
%%capture
!wget https://github.com/ipavlopoulos/xlmr-emotion/raw/main/xlmr-e.weights.h5

In [ ]:
#@title Load the model

# Build the new model
max_len=109
new_model = build_model(transformer_layer, max_len=max_len)

# Load the saved weights
new_model.load_weights('xlmr-e.weights.h5')
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [27]:
#@title Predict for a new text

#Function for encoding according to XLM-R
def encode(texts, tokenizer, maxlen):
    dic = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False, #need only the input_ids
        truncation=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    return np.array(dic['input_ids'])

emotions = ["anger", "anticipation", "disgust", "fear", "joy", "sadness", "surprise", "trust"]
maxlen = 109
def predict(texts, model=new_model, tokenizer=tokenizer, maxlen=109, emotion=emotions):
    encoded_texts = encode(texts, tokenizer, maxlen)
    predictions = model.predict(encoded_texts)
    return [dict(zip(emotions+["none"], list(p)+[1-max(p)])) for p in predictions]

predict(texts=["this is a text"])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 669ms/step


[{'anger': 0.3009773,
  'anticipation': 0.11039856,
  'disgust': 0.33634093,
  'fear': 0.14897384,
  'joy': 0.3111991,
  'sadness': 0.2342824,
  'surprise': 0.044223376,
  'trust': 0.039620537,
  'none': 0.6636590659618378}]